<a href="https://colab.research.google.com/github/NirantK/span_cat_demo/blob/main/spaCy_SmallDataExp_CoNLL2003.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [155]:
%cd /content/
!git clone https://github.com/NirantK/spacy
%cd /content/spacy
!git checkout feature/span-categorizer-v3

/content
fatal: destination path 'spacy' already exists and is not an empty directory.
/content/spacy
Already on 'feature/span-categorizer-v3'
Your branch is up to date with 'origin/feature/span-categorizer-v3'.


In [156]:
!pip install -r requirements.txt
!pip install --no-build-isolation --editable .
!pip install spacy-transformers

Obtaining file:///content/spacy
    Preparing wheel metadata ... done
  Found existing installation: spacy 3.1.0
    Can't uninstall 'spacy'. No files were found to uninstall.
  Running setup.py develop for spacy


In [157]:
import spacy
from spacy.tokens import DocBin, Doc
nlp = spacy.blank("en")

In [158]:
%cd /content/
!git clone https://github.com/NirantK/span_cat_demo

/content
fatal: destination path 'span_cat_demo' already exists and is not an empty directory.


In [159]:
from spacy.util import registry
from typing import List, Callable, Optional, Dict
from thinc.api import Ops, get_current_ops, to_numpy
from thinc.types import Ragged
from pathlib import Path
import numpy as np

In [160]:
# What do I want to do? 
# I want the prediction scores against a specific split: i.e. test.spacy. That has the annotations. 
# For the spans, I can call the ngram suggestion function. 
# This is what the predict function in spancat component does
%cd /content/spacy/
!git pull -a
%cd /content/span_cat_demo/custom
# !python -m spacy project assets
# !git pull -a
# !python -m spacy project run all
!rm -rv ./metrics/spancat_roc.json
!python scripts/roc_evaluate.py ./training/model-best ./corpus/test.spacy ./metrics/spancat_roc.json
!ls ./metrics

/content/spacy
Already up to date.
/content/span_cat_demo/custom
removed './metrics/spancat_roc.json'
2021-06-25 13:45:32.462440: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
<class 'list'> 2 92963 <class 'list'> 92963
cpu_spancat.json  spancat.json	spancat_roc.json


In [161]:
!head ./metrics/spancat_roc.json

{
  "spans":"[[0, 1], [1, 2], [2, 3], [3, 4], [4, 5], [5, 6], [6, 7], [7, 8], [8, 9], [9, 10], [10, 11], [11, 12], [12, 13], [13, 14], [14, 15], [15, 16], [16, 17], [17, 18], [18, 19], [19, 20], [20, 21], [21, 22], [22, 23], [23, 24], [24, 25], [25, 26], [26, 27], [27, 28], [28, 29], [29, 30], [30, 31], [31, 32], [32, 33], [33, 34], [34, 35], [35, 36], [36, 37], [37, 38], [38, 39], [39, 40], [40, 41], [41, 42], [42, 43], [43, 44], [44, 45], [45, 46], [46, 47], [47, 48], [48, 49], [49, 50], [50, 51], [51, 52], [52, 53], [53, 54], [54, 55], [55, 56], [56, 57], [57, 58], [58, 59], [59, 60], [60, 61], [61, 62], [62, 63], [63, 64], [64, 65], [65, 66], [66, 67], [67, 68], [68, 69], [69, 70], [70, 71], [71, 72], [72, 73], [73, 74], [74, 75], [75, 76], [76, 77], [77, 78], [78, 79], [79, 80], [80, 81], [81, 82], [82, 83], [83, 84], [84, 85], [85, 86], [86, 87], [87, 88], [88, 89], [89, 90], [90, 91], [91, 92], [92, 93], [93, 94], [94, 95], [95, 96], [96, 97], [97, 98], [98, 99], [99, 100], [100

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [162]:
# What do I want to do? 
# I've the spans and the corresponding confidence scores
# I've the ground truth in form of doc.ents and their start, end pairs which I can construct again
# What I need is 2 arrays: y_true for all the possible spans, and y_pred for each (span, label) pair

In [163]:
import srsly
import ast
from pathlib import Path

spans_scores_path, loc = Path("./metrics/spancat_roc.json"), Path("./corpus/test.spacy")
assert spans_scores_path.exists() and loc.exists()
data = srsly.read_json(spans_scores_path)

def change_type(info: str)->np.ndarray:
    x = ast.literal_eval(info)
    return np.array(x)

spans, span_lengths, pred_scores = change_type(data["spans"]), change_type(data["span_lengths"]), change_type(data["scores"])
doc_bin = DocBin().from_disk(loc)
docs = list(doc_bin.get_docs(nlp.vocab))

from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
lb.fit(["PER", "LOC", "ORG", "MISC"])

def get_gold_spans(docs: List[Doc])->np.array:
    doc_ent_pos, lengths, labels = [], [], []
    for doc in docs:
        ent_pos = [[ent.start, ent.end] for ent in doc.ents]
        ent_labels = [ent.label_ for ent in doc.ents]
        doc_ent_pos.extend(ent_pos)
        lengths.append(len(ent_pos))
        labels.extend(ent_labels)
    return np.array(doc_ent_pos), np.array(lengths), np.array(labels)

gold_spans, gold_lengths, gold_labels = get_gold_spans(docs)

In [164]:
spans.shape, span_lengths.shape, pred_scores.shape, gold_spans.shape, gold_lengths.shape

((92963, 2), (369,), (92963, 4), (5648, 2), (369,))

In [238]:
suggest_start, ent_start = 0, 0
docs_y_true = []

for suggested_count, entities_count in zip(span_lengths, gold_lengths):
    suggestions = spans[suggest_start: suggest_start+suggested_count]
    suggest_start += suggested_count
    entities = gold_spans[ent_start: ent_start+entities_count]
    labels = gold_labels[ent_start: ent_start+entities_count]
    ent_start += entities_count
    max_overlap = min(suggestions.shape[0], entities.shape[0])
    y_true = []
    assert len(labels) == len(entities)
    assert len(span_lengths) == len(gold_lengths)
    for suggested_span in suggestions:
        flag = False
        for ent, label in zip(entities, labels):
            if not flag:
                flag = (ent == suggested_span).all()
                if flag:
                    y_true.append(label)
                    ent_found = True
        if not flag:
            y_true.append(0)
    docs_y_true.extend(y_true)

docs_y_true = np.array(docs_y_true)
docs_y_true.shape, pred_scores.shape, docs_y_true[3], idx

((92963,), (92963, 4), 'LOC', 266)

((98313,), (92963, 4), 'LOC')

In [ ]:
# Artificially introducing a 4th class: None of the labels, with score 1.0. Since we're using a large number of suggesters, this can fuck up the results. 
# To compensate, we'll calculate all the metrics as One V Rest

In [212]:
label_dict = {
    "LOC": 0, 
    "PER": 1, 
    "MISC": 2, 
    "ORG": 3
}

def sigmoid(x):
    return 1/(1 + np.exp(-x))

sum_one = np.array([sigmoid(x) for x in pred_scores])

for idx, key in enumerate(label_dict.keys()):
    cls_y_true = np.array([1 if element == "LOC" else 0 for element in docs_y_true])

cls_y_score = np.array([x[idx] for x in pred_scores])
len(cls_y_true), len(docs_y_true), len(pred_scores), len(cls_y_score)
# assert len(cls_y_true) == len(cls_y_score)
# roc_auc_score(y_true=cls_y_true, y_score=sum_to_one, average="weighted", multi_class="ovr")

(98313, 98313, 92963, 92963)

In [ ]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

sum_to_one = np.array([softmax(x) for x in pred_scores])

In [179]:
softmax(pred_scores[0])

array([0.24995633, 0.25000353, 0.24996062, 0.25007951])

In [166]:
# from sklearn.datasets import load_iris
# from sklearn.linear_model import LogisticRegression
# X, y = load_iris(return_X_y=True)
# clf = LogisticRegression(solver="liblinear").fit(X, y)
# roc_auc_score(y, clf.predict_proba(X), multi_class='ovr')
# y.shape, set(y)

((150,), {0, 1, 2})

In [154]:
p = clf.predict_proba(X)
p.shape, p[0]

((150, 3), array([8.78030305e-01, 1.21958900e-01, 1.07949250e-05]))

In [146]:
from sklearn.metrics import roc_auc_score
docs_y_true.shape, docs_y_true[3]

((98313,), 'LOC')

In [108]:
np.sum(docs_y_true)

43962

In [ ]:
# use the gold labels to find the upper bound for recall for any suggester
# i have the gold labels from corpus for dev, and test both
# i need the suggester output in some way


def build_ngram_suggester(sizes: List[int]) -> Callable[[List[Doc]], Ragged]:
    """Suggest all spans of the given lengths. Spans are returned as a ragged
    array of integers. The array has two columns, indicating the start and end
    position."""

    def ngram_suggester(docs: List[Doc], *, ops: Optional[Ops] = None) -> Ragged:
        if ops is None:
            ops = get_current_ops()
        spans = []
        lengths = []
        for doc in docs:
            starts = ops.xp.arange(len(doc), dtype="i")
            starts = starts.reshape((-1, 1))
            length = 0
            for size in sizes:
                if size <= len(doc):
                    starts_size = starts[:len(doc) - (size - 1)]
                    spans.append(ops.xp.hstack((starts_size, starts_size + size)))
                    length += spans[-1].shape[0]
                if spans:
                    assert spans[-1].ndim == 2, spans[-1].shape
            lengths.append(length)
        if len(spans) > 0:
            output = Ragged(ops.xp.vstack(spans), ops.asarray(lengths, dtype="i"))
        else:
            output = Ragged(ops.xp.zeros((0,0)), ops.asarray(lengths, dtype="i"))

        assert output.dataXd.ndim == 2
        return output

    return ngram_suggester

n_gram_suggester = build_ngram_suggester(sizes = [1, 2, 3])

In [ ]:
doc_bin = DocBin().from_disk(loc)
docs = list(doc_bin.get_docs(nlp.vocab))

In [ ]:
# cp
from collections import Counter
ent_length = []
for doc in docs: 
    l = [len(ent) for ent in doc.ents]
    ent_length.extend(l)
print(Counter(ent_length))

In [ ]:
def get_gold_spans(docs: List[Doc])->np.array:
    doc_ent_pos, lengths = [], []
    for doc in docs:
        ent_pos = [[ent.start, ent.end] for ent in doc.ents]
        doc_ent_pos.extend(ent_pos)
        lengths.append(len(ent_pos))
    return np.array(doc_ent_pos), np.array(lengths)

gold_spans, lengths = get_gold_spans(docs)
ngram_pos_list = ngram_suggester(docs)

In [ ]:
assert ngram_pos_list.lengths.shape == lengths.shape
ngram_pos_list.data.shape, gold_spans.shape, lengths.shape, ngram_pos_list.lengths.shape
# type(gold_spans[0])

In [ ]:
suggest_start, ent_start = 0, 0
doc_overlaps = []
for suggest_cnt, ent_cnt in zip(ngram_pos_list.lengths, lengths):
    suggestions = ngram_pos_list.data[suggest_start: suggest_start+suggest_cnt]
    suggest_start += suggest_cnt
    entities = gold_spans[ent_start: ent_start+ent_cnt]
    ent_start += ent_cnt
    assert len(entities) ==  ent_cnt
    assert len(suggestions) == suggest_cnt
    # print(len(suggestions), len(entities))
    A, B = entities, suggestions
    # We can create a boolean mask using broadcasting which can be then used to filter the rows in array A which are also present in array B
    # mask = (A[:, None] == B).all(-1).any(1)
    # overlap = A[mask]
    # Alternatively, we can use native Python set tuples
    aset = set([tuple(x) for x in A])
    bset = set([tuple(x) for x in B])
    # print(len(aset), len(bset), aset, bset)
    overlap = aset & bset
    # overlap - np.array([x for x in aset & bset])
    # assert len(overlap) < len(suggestions)
    # assert len(overlap) < len(entities)
    doc_overlaps.append({"intersect": len(overlap), "doc_r": len(overlap) / ent_cnt, "ent_cnt": ent_cnt})

len(doc_overlaps)

In [ ]:
total_ents_found, avg_recall = 0, 0.0
for element in doc_overlaps:
    total_ents_found += element["intersect"]
    if element["ent_cnt"] > 0:
        avg_recall += element["doc_r"]

print(f"Global Recall (Calculated as Total Correct Suggestions/Total Entities): {total_ents_found/gold_spans.shape[0]:.4f}")
print(f"Average (Calculated on a Per Doc Level): {avg_recall/len(doc_overlaps):.4f}")